In [1]:
import os
import fnmatch

def buscar_archivos_nii_gz(directorio, cadena):
    archivos_con_cadena = []

    # Recorrer todos los directorios y archivos en el directorio dado
    for raiz, carpetas, archivos in os.walk(directorio):
        for archivo in fnmatch.filter(archivos, '*' + cadena + '*'):
            ruta_completa = os.path.join(raiz, archivo)
            archivos_con_cadena.append(ruta_completa)

    return archivos_con_cadena

# Ejemplo de uso
directorio_a_buscar_control = 'duenhas/Preprocessed/control'
directorio_a_buscar_parkinson = 'duenhas/Preprocessed/parkinson'
cadena_a_buscar = 'brain_mask_applied.nii.gz'
control = buscar_archivos_nii_gz(directorio_a_buscar_control, cadena_a_buscar)
parkison = buscar_archivos_nii_gz(directorio_a_buscar_parkinson, cadena_a_buscar)


# Imprimir la lista de archivos encontrados
print("Archivos control encontrados:", len(control))
print("Archivos parkinson encontrados:", len(parkison))


Archivos control encontrados: 16
Archivos parkinson encontrados: 27


In [11]:
from nilearn.maskers import nifti_labels_masker
import matplotlib.pyplot as plt
import nibabel as nib
from nilearn import plotting as nlp
import pandas as pd
import seaborn as sns
import numpy as np

In [14]:
import nibabel as nib
import numpy as np
from sklearn.model_selection import train_test_split

# Listas de rutas para los grupos "parkinson" y "control"


# Función para cargar las imágenes
def cargar_imagenes(lista_rutas):
    imagenes = []
    for ruta in lista_rutas:
        img = nib.load(ruta)
        datos = img.get_fdata()
        imagenes.append(datos)
    return imagenes

# Cargar las imágenes para el grupo "parkinson"
imagenes_parkinson = cargar_imagenes(parkison)
etiquetas_parkinson = np.ones(len(imagenes_parkinson))  # Etiqueta 1 para "parkinson"

# Cargar las imágenes para el grupo "control"
imagenes_control = cargar_imagenes(control)
etiquetas_control = np.zeros(len(imagenes_control))  # Etiqueta 0 para "control"

# Combina los datos y etiquetas de ambos grupos
conjunto_datos = np.concatenate([imagenes_parkinson, imagenes_control])
etiquetas = np.concatenate([etiquetas_parkinson, etiquetas_control])

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
conjunto_entrenamiento, conjunto_prueba, etiquetas_entrenamiento, etiquetas_prueba = train_test_split(
    conjunto_datos, etiquetas, test_size=0.2, random_state=42
)

# Ahora puedes utilizar conjunto_entrenamiento y etiquetas_entrenamiento para entrenar tu CNN
# y conjunto_prueba y etiquetas_prueba para evaluar su rendimiento.

# Asegúrate de adaptar las rutas y etiquetas según tu estructura de datos específica.



In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout

# Definir el modelo de la CNN
modelo = Sequential()

# Capa de convolución 3D
modelo.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=(tamaño_x, tamaño_y, tamaño_z, canales)))
# Puedes agregar más capas de convolución y max pooling según sea necesario

# Capa de max pooling 3D
modelo.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Aplanar los datos para la capa completamente conectada
modelo.add(Flatten())

# Capas densas (completamente conectadas)
modelo.add(Dense(128, activation='relu'))
modelo.add(Dropout(0.5))  # Agregar dropout para reducir el sobreajuste
modelo.add(Dense(1, activation='sigmoid'))  # Salida binaria, puedes ajustar según tu tarea

# Compilar el modelo
modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Resumen del modelo
modelo.summary()


2023-11-10 21:03:28.700454: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-10 21:03:29.692943: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-10 21:03:29.692974: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-10 21:03:29.701704: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-10 21:03:30.320908: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-10 21:03:30.328741: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

NameError: name 'tamaño_x' is not defined